<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Knowledge Retrieval - один файл в сообщении

Давайте разберем концепцию Knowledge Retrieval в более образовательном и объяснительном тоне:

**Что такое Knowledge Retrieval?**
Knowledge Retrieval - это функция, которая расширяет возможности AI Assistant за счет интеграции внешних источников знаний. Это может быть конкретная информация, связанная с продуктом, или документы, предоставленные пользователями.

**Как это работает?**
Когда вы загружаете документ в AI Assistant, OpenAI обрабатывает его в несколько этапов:

1. **Разбивка и индексирование**: Документ делится на более мелкие чанки ("куски") и индексируется, то есть организуется таким образом, чтобы ИИ мог эффективно в нем искать.

2. **Хранение Встраиваний**: Встраивания - это как бы сжатое представление содержимого документа, фиксирующее его важную информацию в формате, который ИИ может понять и извлечь при необходимости.

3. **Внедрение векторного поиска**: Это метод, который ИИ использует для поиска наиболее релевантных частей документа на основе полученного запроса.

**Включение поиска для помощника**
При активации поиска для определенного помощника ИИ все прикрепленные файлы автоматически обрабатываются, как описано выше. Однако эта функция стоит 0,10 доллара за гигабайт в день (ДО 1 ГБ бесплатно) для каждого помощника, у которого включено извлечение.

**Изменение настроек помощника**
Вы можете включить или выключить функцию извлечения с помощью конечной точки Modify Assistant, которая является частью API Assistant.

**Техника извлечения**
ИИ-помощник выбирает один из двух методов поиска в зависимости от сообщений пользователя:

1. ***Прямое включение подсказки**: Для коротких документов ИИ включает содержимое непосредственно в ответную подсказку.

2. **Векторный поиск**:  Для более длинных документов он выполняет векторный поиск, чтобы найти наиболее релевантную информацию.

 
В настоящее время система ставит во главу угла качество, включая весь релевантный контент в контекст модели для ответа на запросы. Однако OpenAI планирует внедрить больше стратегий поиска, позволяя разработчикам выбирать баланс между качеством поиска и стоимостью использования модели. 
 

In [1]:
from openai import OpenAI
import json
client = OpenAI()

----

## Uploading Files
### 1) Загрузка файлов через прямое указание их путей

In [13]:
import os
files_to_upload = ['plan_upay.docx', 'plan_in_sign.docx', 'signs.docx']

## Функция загрузки файлов сначало на сервер OpenAI с указанием что эти файлы предназначены для Assistant

In [14]:
def upload_assistant_file(filename):
    file = client.files.create(
      file=open(filename, "rb"),
      purpose='assistants'
    )
    print(file.id)

## Запуск функции загрузки файлов на сервер. 

In [15]:
for file in files_to_upload:
    upload_assistant_file(file)

file-8wFAh1pAqigNn8JX5c4ZNUvd
file-ojV1fuoUcfGIeoMZLLCAViaK
file-9nq5UX3m6hJELK6uZSsaZT6K


## Проверка имен файлов и их id

In [16]:
response = client.files.list()
for file in response.data:
    print(f"Имя файла: {file.filename}, ID файла: {file.id}")

Имя файла: signs.docx, ID файла: file-9nq5UX3m6hJELK6uZSsaZT6K
Имя файла: plan_in_sign.docx, ID файла: file-ojV1fuoUcfGIeoMZLLCAViaK
Имя файла: plan_upay.docx, ID файла: file-8wFAh1pAqigNn8JX5c4ZNUvd


## Далее создаем векторную базу и помещаем в нее файлы которые мы загрузили

## СОздали ВекБД

In [29]:
vector_store = client.beta.vector_stores.create(
  name="Астрологические интерпритации"
)

## Вспомогательная функция которая выводит список id ВекБД и имя ВекБД

In [29]:
def get_vector_store_ids_and_names(vector_stores_response):
    """
    Извлекает список ID и имен хранилищ векторов из SyncCursorPage.

    Args:
        vector_stores_response: Объект SyncCursorPage возвращаемый API-вызовом vector_stores.list().

    Returns:
        Список словарей, где каждый словарь содержит ID и имя хранилища векторов.
    """

    vector_store_info = []
    for store in vector_stores_response.data:
        vector_store_info.append({"id": store.id, "name": store.name})
    return vector_store_info


## Вызов вспомогательной функции

In [30]:
vector_stores = client.beta.vector_stores.list()
vector_store_ids = get_vector_store_ids_and_names(vector_stores)
print(vector_store_ids)

[{'id': 'vs_tKHpFBZxd6x7lPX8WkiykuCO', 'name': 'Астрологические интерпритации'}]


## Удаление выбранной ВекБД по id

In [27]:
deleted_vector_store = client.beta.vector_stores.delete(
  vector_store_id="vs_GKW6jzMv7T1ep9rAleh2cW4R"
)
print(deleted_vector_store)

VectorStoreDeleted(id='vs_GKW6jzMv7T1ep9rAleh2cW4R', deleted=True, object='vector_store.deleted')


## Загрузка файлов пакетом, из уже загруженных файлов на сервере OpenA, где далее мы помещаем их в уже созданную век.бд файлы пакетом. То есть мы загрузили какие то файлы на OpenAI, далее мы взяли id определенных файлов, сформировали пакет и загрузили в ВекБД

## Вспомогательная функция для загрузки файлов пакетом в ВЕКБД

In [10]:
from datetime import datetime

def load_files_to_vector_store(vector_store_id, file_ids):
    """
    Загружает файлы в хранилище векторов.

    Args:
        vector_store_id: ID хранилища векторов.
        file_ids: Список ID файлов.

    Returns:
        Объект ответа от API 'vector_stores.file_batches.create'.
    """
    batch_name = f"Загрузка файлов - {datetime.now().isoformat()}"
    print(f"Начинаем загрузку файлов в хранилище векторов '{vector_store_id}' под названием '{batch_name}'.")

    batch = client.beta.vector_stores.file_batches.create(
        vector_store_id=vector_store_id,
        file_ids=file_ids
    )

    print(batch_name)
    print(f"Статус загрузки: {batch.status}")
    print(f"Количество файлов в пакете: {batch.file_counts.total}")
    return batch


## Здесь Запускаем вспомогательную функцию, и определяем id ВекБД и id файлов которые будут в пакете

In [17]:
vector_store_id = "vs_tKHpFBZxd6x7lPX8WkiykuCO"  # Замените на ID вашего хранилища векторов
file_ids = [
    "file-8wFAh1pAqigNn8JX5c4ZNUvd",
    "file-ojV1fuoUcfGIeoMZLLCAViaK",
    "file-9nq5UX3m6hJELK6uZSsaZT6K"
]
batch = load_files_to_vector_store(vector_store_id, file_ids)

Начинаем загрузку файлов в хранилище векторов 'vs_tKHpFBZxd6x7lPX8WkiykuCO' под названием 'Загрузка файлов - 2024-05-05T12:09:33.013091'.


Загрузка файлов - 2024-05-05T12:09:33.013091
Статус загрузки: in_progress
Количество файлов в пакете: 3
Ошибка загрузки: in_progress


## Вспомогательная функция для проверки для проверки выполнения загрузки пакета файлов в векБД

In [38]:
def check_file_batch_status(vector_store_id, batch_id):
  """
  Проверяет статус загрузки файлов в хранилище векторов.

  Args:
    vector_store_id: ID хранилища векторов.
    batch_id: ID пакета загрузки файлов.

  Returns:
    Статус загрузки (например, "completed", "in_progress", "failed").
  """

  vector_store_retrieve_file_batch = client.beta.vector_stores.file_batches.retrieve(
      vector_store_id=vector_store_id,
      batch_id=batch_id
  )

  return vector_store_retrieve_file_batch.status

# Пример использования
vector_store_id = "vs_tKHpFBZxd6x7lPX8WkiykuCO"
batch_id = "vsfb_59c289e2cfac44f0a2861accf947e44c"

status = check_file_batch_status(vector_store_id, batch_id)

if status == "completed":
  print("Файлы успешно загружены!")
else:
  print(f"Результат: {status}")

Файлы успешно загружены!


## Adding Files for the Assistant (or in a message)

Let's explain the process of uploading files for retrieval in a more educational and explanatory way:

**Uploading Files for Retrieval: An Overview**
When you want to use external documents to enhance the AI Assistant's responses, you can upload these files for retrieval. This process is somewhat similar to how the Code Interpreter works.

**Two Levels of Attachment**

1. **Assistant-Level Attachment**: When you attach a file at this level, it becomes a part of the Assistant's general knowledge base. This means the Assistant can access this information when responding to any query.

2. **Message-Level Attachment**: In contrast, when you attach a file at the Message-level, it's only accessible for the specific conversation thread (or 'Thread') that the message is a part of. This is more targeted, as the file's content will only be used to respond to queries within that particular thread.

**How to Attach a File to a Message**
After uploading a file, you will receive a unique ID for that file. You can then use this ID to attach the file to a specific message. This way, the AI knows which document to refer to when crafting its response.

**Charges for File Usage**
It's important to note that the cost isn't based on the size of the files you upload. Instead, the charges are based on which files you attach to an Assistant or a Message that then get indexed (processed and made searchable by the AI).

**File Size and Format Limitations**
There are some limits on what you can upload:

- **Maximum File Size**: Each file can be up to 512 MB.
- **Token Limit**: The content of the file should not exceed 2,000,000 tokens (a token is roughly a word or a piece of a word, and this limit is calculated automatically when you attach the file).
- **Supported Formats**: The retrieval system is compatible with various file formats, including popular ones like PDF (.pdf), Markdown (.md), and Word documents (.docx).

In summary, by uploading files at either the Assistant-level or Message-level, you can significantly enhance the AI Assistant's ability to provide detailed and relevant responses based on the content of these documents. This feature supports various file formats and sizes, with costs incurred based on the indexing of attached files.

In [40]:
# Add the file to the assistant
assistant = client.beta.assistants.create(
  name='Бот ведической астрологии',
  instructions="Вы отвечаете развернуто по вопросам ведической астрологии, ваш ответ строиться строго на предоставленных вам текстовых документах",
  model="gpt-3.5-turbo-0125", 
  tools=[{"type": "file_search"}], # Список инстументов которые может использовать помощник
  tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}} 
  """
  tool_resources -->
  Ресурсы которые помощник будет использовать в своих инструментах (То есть мы можем 
  определять не только инструменты, но и ресурсы которые будут в этих инструментах. Конктретно в нашем примере мы
  четко задали что бы помощник использовал)
  """
)

## Run a Thread and Add a File to Message

When a file is attached at the Message-level, it is only accessible within the specific Thread the Message is attached to. After having uploaded a file, you can pass the ID of this File when creating the Message. Note that you are not charged based on the size of the files you upload via the Files API but rather based on which files you attach to a specific Assistant or Message that get indexed.

In [41]:
thread = client.beta.threads.create(
    messages=[
    {
      "role": "user",
      "content": "Здравстуйте, вам сейчас будут заданы вопросы об астрологии.",
    }
  ]
)

In [81]:
thread_1 = client.beta.threads.update(
  thread_id=thread.id,
  metadata={
    "Предназначение": "Астрология",
    "user": "maks"
  }
)

In [87]:
print(json.dumps(thread_1.to_dict(), indent=2, ensure_ascii=False))

{
  "id": "thread_jqhZgocR7vbvLXajrtiesqj0",
  "created_at": 1714903923,
  "metadata": {
    "Предназначение": "Астрология",
    "user": "maks"
  },
  "object": "thread",
  "tool_resources": {}
}


In [42]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Какие упайи нужны для Солнца?"
    
)

In [43]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  additional_instructions= "Добавьте в конце вашего ответа (Надеюсь мой ответ помог вам, если у вас есть дполнительные вопросы то задавайте)",
  tool_choice= "required" # Команда required Указывает модели то что она обязательно должны воспользоваться инструментом
  # которые мы определили при создании assistant. 
)

In [56]:
run.status

'completed'

In [57]:
run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )
    
print(run.status)

completed


Once the run is completed, let's get the messages

In [58]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)

In [ ]:
print(json.dumps(messages.to_dict(), indent=2, ensure_ascii=False))

In [63]:
def display_thread_messages(messages):
    # EXPECTS MESSAGES IN ASC ORDER!
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')

In [64]:
display_thread_messages(messages)

Здравстуйте, вам сейчас будут заданы вопросы об астрологии.


Какие упайи нужны для Солнца?


Для усиления и улучшения позиции Солнца, рекомендуется выполнять следующие упайи:

1. Чтение мантры "ом намо бхагавате раманачандрайя" 108 раз с утра по воскресеньям.
2. Вставать до восхода солнца.
3. Употребление масла гхи.
4. Пост в воскресенье.
5. Выполнение комплекса упражнений сурья намаскар.
6. Ухаживать за отцом и показывать уважение к нему.
7. Почитание и уважение отца, начальника, правителя【5:0†source】.

Надеюсь мой ответ помог вам, если у вас есть дополнительные вопросы, не стесняйтесь их задавать!




In [67]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [91]:
print(json.dumps(run_steps.to_dict(), indent=2))

{
  "data": [
    {
      "id": "step_D098Ky4a1A2uXsUkAbgNpo6V",
      "assistant_id": "asst_yOEj5bDErckIQmWM0zgMdSVj",
      "cancelled_at": null,
      "completed_at": 1714906321,
      "created_at": 1714906317,
      "failed_at": null,
      "last_error": null,
      "object": "thread.run.step",
      "run_id": "run_LsLvhchvbkc3PgR8R662hmJn",
      "status": "completed",
      "step_details": {
        "message_creation": {
          "message_id": "msg_kANy1vY18MleOSqMQ0bAojp1"
        },
        "type": "message_creation"
      },
      "thread_id": "thread_jqhZgocR7vbvLXajrtiesqj0",
      "type": "message_creation",
      "usage": {
        "completion_tokens": 252,
        "prompt_tokens": 4993,
        "total_tokens": 5245
      },
      "expires_at": null
    },
    {
      "id": "step_kmRtEagY4DveUmXbbjpkhq7z",
      "assistant_id": "asst_yOEj5bDErckIQmWM0zgMdSVj",
      "cancelled_at": null,
      "completed_at": 1714906317,
      "created_at": 1714906316,
      "failed_at": 

In [69]:
run_step = client.beta.threads.runs.steps.retrieve(
    thread_id=thread.id,
    run_id=run.id,
    step_id="step_D098Ky4a1A2uXsUkAbgNpo6V"
)

In [ ]:
print(json.dumps(run_step.to_dict(), indent=2))

## Вспомогающая функция которая выдает списки

In [4]:
def print_object_info(thread_id, run_id):
    my_assistants = client.beta.assistants.list(
        order="asc"
    )
    thread_messages = client.beta.threads.messages.list(thread_id)
    runs = client.beta.threads.runs.list(thread_id)
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread_id,
        run_id=run_id
    )
    vector_stores = client.beta.vector_stores.list()
    files = client.files.list()

    print("Assistants:")
    for assistant in my_assistants.data:
        print(f"ID: {assistant.id}, Name: {assistant.name}")
    
    print("\nThread Messages:")
    for message in thread_messages.data:
        print(f"ID: {message.id}, Thread ID: {message.thread_id}")
    
    print("\nRuns:")
    for run in runs.data:
        print(f"ID: {run.id}, Thread ID: {run.thread_id}")
    
    print("\nRun Steps:")
    for step in run_steps.data:
        print(f"ID: {step.id}, Thread ID: {step.thread_id}")
        
    print("\nVector stores:")
    for vector in vector_stores.data:
        print(f"ID: {vector.id}, Name: {vector.name}, Files: {vector.file_counts}")
        
    print("\nFiles:")
    for file in files.data:
        print(f"ID: {file.id}, File name: {file.filename}, Purpose: {file.purpose}")
        
    

In [ ]:
# Вызов функции с соответствующими аргументами
print_object_info(thread_id=thread.id, run_id=run.id)

In [7]:
my_assistants = client.beta.assistants.list(
        order="asc"
    )
print("Assistants:")
for assistant in my_assistants.data:
    print(f"ID: {assistant.id}, Name: {assistant.name}")

Assistants:
ID: asst_yOEj5bDErckIQmWM0zgMdSVj, Name: Бот ведической астрологии
ID: asst_id9UIqBRt68Ucthq2VJ7J80s, Name: Stock Visualizer
ID: asst_5wao97wKjIYCw2l0g0nS4cxI, Name: помощник
ID: asst_OywtDLNTG9xNIr9uzgBqIxdo, Name: Stock Visualizer


# Optional: Delete Assistant and All Files

In [94]:
def delete_resource(resource_type, resource_id, vector_store_id=None):
    try:
        if resource_type == "assistant":
            response = client.beta.assistants.delete(resource_id)
            print(f"Assistant '{resource_id}' успешно удален.")
        elif resource_type == "thread":
            response = client.beta.threads.delete(resource_id)
            print(f"Thread '{resource_id}' успешно удален.")
        elif resource_type == "vector_store":
            deleted_vector_store = client.beta.vector_stores.delete(vector_store_id=resource_id)
            print(f"Vector Store '{resource_id}' успешно удален.")
        elif resource_type == "vector_store_file":
            if vector_store_id is None:
                raise ValueError("Для удаления Vector Store File необходимо указать 'vector_store_id'.")
            deleted_vector_store_file = client.beta.vector_stores.files.delete(
                vector_store_id=vector_store_id,
                file_id=resource_id
            )
            print(f"Vector Store File '{resource_id}' успешно удален из Vector Store '{vector_store_id}'.")
        else:
            raise ValueError(f"Неверный тип ресурса: {resource_type}. Допустимые значения: 'assistant', 'thread', 'vector_store', 'vector_store_file'.")
    except Exception as e:
        print(f"Ошибка при удалении ресурса: {str(e)}")

In [8]:
response = client.files.list()
for file in response.data:
    print(f"Имя файла: {file.filename}, ID файла: {file.id}")

Имя файла: Corporate Travel Policy.pdf, ID файла: file-UDDeMWox3gSsXhI6PCYcPpzv
Имя файла: ACME_Dog_Food_Ingredients.pdf, ID файла: file-J6Bb1Sff4EawtGOcSWwnS1da
Имя файла: ACME Advertising Policy.pdf, ID файла: file-1usqsdyqYKodrqyLGG8zibzk
Имя файла: 065cfd7c-1ca5-498b-9bff-6127a39b1d23, ID файла: file-Daq3frspraqVsXRw0OyRQb0k
Имя файла: SP500_Prices_5Year.csv, ID файла: file-VYCIXgLhrqdn5sOuBRd1tMHD
Имя файла: signs.docx, ID файла: file-9nq5UX3m6hJELK6uZSsaZT6K
Имя файла: plan_in_sign.docx, ID файла: file-ojV1fuoUcfGIeoMZLLCAViaK
Имя файла: plan_upay.docx, ID файла: file-8wFAh1pAqigNn8JX5c4ZNUvd


In [ ]:
# Удаление Assistant
delete_resource(resource_type="assistant", resource_id="asst_abc123")

In [ ]:
# Удаление Thread
delete_resource(resource_type="thread", resource_id="thread_abc123")

In [98]:
# Удаление Vector Store
delete_resource(resource_type="vector_store", resource_id="vs_tKHpFBZxd6x7lPX8WkiykuCO")

Vector Store 'vs_tKHpFBZxd6x7lPX8WkiykuCO' успешно удален.


In [ ]:
# Удаление Vector Store File
delete_resource(resource_type="vector_store_file", resource_id="file-abc123", vector_store_id="vs_abc123")